In [1]:
from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from mkdtree import kdtree

"""Initializes the number of nodes and generates the genesis block"""

print('START')
MAX_NODES = 9999
num_nodes = 80

node_id_list = [int(MAX_NODES / 2)]
tree = kdtree.create_root(4, 40)

print(tree)
print(tree.sel_axis)

START


TypeError: create_root() missing 1 required positional argument: 'forger'

In [ ]:
"""Sets the initial clusters based on the data file"""

#changes listeners subscription from one cluster to another.
def move_listener(listener, old_topic, new_topic):
    pub.unsubscribe(listener, old_topic) #core.TopicManager.getTopicsSubscribed(listener))
    pub.subscribe(listener, new_topic)

#Opens the data set file and generates nodes based on the provided data.
#file is comma delimited in the following format: time-nodeID-xcoor-ycoor-miner-region-prevregion
with open('node_data.txt') as f:
    lines = f.readlines() # list containing lines of file
    i = 0
    node = []
    for line in lines:
        parts = line.split(',')
        #itterates through based on the number of nodes provided.
        if i < num_nodes:
            new_node = Node(i + 1, parts[5])
            new_node.startListener("c" + str(parts[5]).strip())
            node.append(new_node)
        else:
            move_listener(node[i % num_nodes].node_listener, "c" + str(parts[6]).strip(), "c" + str(parts[5]).strip())
        i += 1

In [ ]:
print('PRE-LOOP')
while True:
    print('IN-LOOP')
    selected_node = random.randint(0, num_nodes)
    if selected_node not in node_id_list:
        node_id_list.append(selected_node)
    if len(node_id_list) == num_nodes:
        break
print('POST-LOOP')

print(node_id_list)

In [ ]:
print(f'node_id_list size: {len(node_id_list)}')
print(f'node_id_list: {node_id_list}')

# key_pair already exists in Wallet.py
wallets = {}
for n in node_id_list:
    wallets[n] = Wallet()
    # key_pairs[n] = RSA.generate(2048)


In [ ]:
FORGE_POINT = 3

chain1 = MKDBlockchain()
chain2 = MKDBlockchain()

pool1 = TransactionPool()
pool2 = TransactionPool()


def create_transaction(sender, receiver, pool, amount, tr_type):
    transaction = Transaction(sender, receiver, amount, tr_type)
    if not pool.transaction_exists(transaction):
        pool.add_transaction(transaction)

def create_transaction_batch(pool, t_type, t_num):
    if pool1.forging_required():
        forger = list(wallets.values())[(random.randint(0,1000)%10)]
        chain1.add_block(chain1.create_block(pool1.transactions, forger))
        pool1.remove_from_pool(pool1.transactions)
    for i in range(0,t_num):
        # create_transaction(sender, receiver, pool, random.randint(0, 500), t_type)
        sender = list(wallets.values())[(random.randint(0,1000)%10)].public_key_string()
        receiver = list(wallets.values())[(random.randint(0,1000)%10)].public_key_string()

        create_transaction(sender, receiver, pool, random.randint(0, 500), t_type)


create_transaction_batch(pool1, 'EXCHANGE', 1000)
create_transaction_batch(pool2, 'EXCHANGE', 20000)

create_transaction(list(wallets.values())[random.randint(0,1000)%4].public_key_string(),
                   list(wallets.values())[random.randint(0,1000)%4].public_key_string(),
                   pool2, 226, 'TRANSFER')

ts = [t.to_json() for t in pool1.transactions]

# pprint.pprint(ts)
# pprint.pprint(pool1.transactions)
pprint.pprint(chain1.to_json())


print(chain1)
print('END')
